<a href="https://colab.research.google.com/github/gabrielhozana/imdb-synopsis-indonesian-movies/blob/main/Multiclass_Text_Classification_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Menggunakan dataset yang berisi sinopsis dari beberapa film Indonesia dan genrenya. Tujuannya adalah untuk menentukan genre sebuah film berdasarkan sinopsisnya. 

In [1]:
!pip install kaggle
# https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb#scrollTo=hMY4CFezjcG-
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


In [2]:
# https://www.kaggle.com/antoniuscs/imdb-synopsis-indonesian-movies
!kaggle datasets download -d antoniuscs/imdb-synopsis-indonesian-movies

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 44.7MB/s]


In [3]:
!unzip imdb-synopsis-indonesian-movies.zip

Archive:  imdb-synopsis-indonesian-movies.zip
  inflating: imdb_indonesian_movies_2.csv  


In [5]:
import pandas as pd

df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

In [6]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [7]:
#ubah label kategorikal 
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [8]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [16]:
print(label)

[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]]


In [10]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
     
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
     
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
     
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [21]:
# https://github.com/tensorflow/tensorflow/issues/23698
# increase the vocabulary size.
# It is 5000 by default. This error is like an out of bound index error.
#using len(tokenizer.word_index)+1 for increase the vocabulary size
import tensorflow as tf
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=16),
                             tf.keras.layers.LSTM(64),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(5, activation='softmax')
                             ])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [22]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs,validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 108s - loss: 1.6101 - accuracy: 0.1903 - val_loss: 1.6113 - val_accuracy: 0.1741
Epoch 2/30
26/26 - 27s - loss: 1.6064 - accuracy: 0.2214 - val_loss: 1.6095 - val_accuracy: 0.1841
Epoch 3/30
26/26 - 27s - loss: 1.9087 - accuracy: 0.2562 - val_loss: 1.6283 - val_accuracy: 0.1841
Epoch 4/30
26/26 - 27s - loss: 1.5878 - accuracy: 0.2239 - val_loss: 1.5999 - val_accuracy: 0.2040
Epoch 5/30
26/26 - 27s - loss: 1.4713 - accuracy: 0.3657 - val_loss: 1.5852 - val_accuracy: 0.2537
Epoch 6/30
26/26 - 27s - loss: 1.1842 - accuracy: 0.4614 - val_loss: 1.7372 - val_accuracy: 0.2189
Epoch 7/30
26/26 - 28s - loss: 0.9340 - accuracy: 0.6194 - val_loss: 2.2931 - val_accuracy: 0.2139
Epoch 8/30
26/26 - 28s - loss: 0.6999 - accuracy: 0.7711 - val_loss: 2.6251 - val_accuracy: 0.2388
Epoch 9/30
26/26 - 27s - loss: 0.4445 - accuracy: 0.8495 - val_loss: 3.1894 - val_accuracy: 0.2289
Epoch 10/30
26/26 - 27s - loss: 0.2310 - accuracy: 0.9316 - val_loss: 3.7514 - val_accuracy: 0.2687
Epoch 11